## import

In [1]:
# auto-reload for debugging
%load_ext autoreload

In [2]:
# auto-reload for debugging
# importing target
%aimport utils.search_address

In [3]:
import pandas as pd 
import numpy as np
import math
import re

from icecream import ic  # for debugging instead of using print
from tqdm.notebook import tqdm 

from utils.custom_utils import load_var, save_var
from utils.webscraping import ArticleScraper
from utils.search_address import CheckAddressLevels
import utils.search_address as sa

from importlib import reload

# Korea Address Information
df_addr = pd.read_parquet('./address/df_addr.parquet.gzip')
# Web-scraped Dataframe
df_text = pd.read_parquet('./dataframe/flasher_hk_20130101_20220307.gzip')


In [4]:
# auto-reload option
%autoreload 2

## searching address in the text

In [5]:
# index = 0
# text_iter = iter(zip(df_text.index[index:], df_text.loc[index:, 'content']))

In [6]:
# index, text = next(text_iter)
# text = sa.clean_text(text)

In [10]:
# the web-scraped dataframe deep copy for test: only 2 features
df_work = df_text.iloc[:2]

In [11]:
# start index and end index
start_index = 0
end_index = len(df_text)

# total length of the dataframe
total_length = len(df_work.iloc[start_index:end_index])

# loop
for index, row in tqdm(df_work[start_index:end_index].iterrows(), total=total_length):
    # loading article content
    text = row['content']
    # checking if the type of the loaded text is string and not empty
    if isinstance(text, str) and len(text) > 0:
        # cleaning the text like strip()
        text = sa.clean_text(text)

        # Class instance - CheckAddressLevels
        # param: the address dataframe (dataframe)
        # param: the loaded article content (string)
        self = CheckAddressLevels(df_address=df_addr, text=text)
        # processing to extract address
        # return value: string
        result = self.process()
        # putting the result into the location column
        df_work.at[index, '지역'] = result

        # marking the location information in the article content
        text_mark = text
        for item in self.df_brief['found_name'].values:
            text_mark = sa.re_sub(item, text_mark)
        # putting the result into the 'found_text' column
        df_work.at[index, 'found_text'] = text_mark

        # extracting sentences having the location only 
        text_marked = sa.extract_sentences_marked(marked_text=text_mark)
        # putting the result into the 'found_sentences' column
        df_work.at[index, 'found_sentences'] = text_marked

  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
# save the dataframe for later 
df_work.to_csv("./dataframe/geo_02_df_work.csv", index=False)

## coordinates

In [14]:
# total length of the processed dataframe
total_length = len(df_work)

# loop
for index, row in tqdm(df_work.iterrows(), total=total_length):
    # get location information using geopy api
    lat, long = sa.get_lat_long(row['지역'])
    # putting latitude and longitude into each columns
    df_work.at[index, '위도'] = lat
    df_work.at[index, '경도'] = long

  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
df_work[['지역', '위도', '경도']]

,지역,위도,경도
0,서울,37.554555,126.970779
1,서울 서울특별시,37.554555,126.970779
